Evan-1103204160, Ini adalah SneakPeak TA saya, saya ingin membuat model untuk prediksi transaksi, yang dibawah adalah dataset transaksi 3 bulan kebelakang, saya melakukan beberapa visualisasi dan mendapatkan beberapa informasi untuk mencari estimator atau atribut dari prediksi transaksi saya.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, IntSlider
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import matplotlib.dates as mdates


In [2]:
file_path="datasets/dataset_wates.xlsx"
import pandas as pd
# Read the Excel file, specifying the 'na_values' parameter to handle missing values
df = pd.read_excel(file_path)
df.head()

,No. Struk,Tanggal,Jam,Nama Outlet,Nama Kasir,Produk,Jumlah Produk,Jumlah Dibatalkan,Harga Per Produk,Subtotal,Tipe Harga,Diskon Produk,Diskon Transaksi,Pajak,Total,Status,Metode Pembayaran
0,5911206E,18-10-2023,12:45:58,Wates,Balibul Wates,"Sate Solo 10 Campur , Reguler",6,0,60000,360000,Normal,0,0,0,360000,Transaksi,CASH
1,5911206E,18-10-2023,12:45:58,Wates,Balibul Wates,"Tongseng , Reguler",1,0,35000,35000,Normal,0,0,0,35000,Transaksi,CASH
2,5911206E,18-10-2023,12:45:58,Wates,Balibul Wates,"Tengkleng Masak/Segar, Reguler",1,0,35000,35000,Normal,0,0,0,35000,Transaksi,CASH
3,5911206E,18-10-2023,12:45:58,Wates,Balibul Wates,"Sop Kambing, Reguler",1,0,35000,35000,Normal,0,0,0,35000,Transaksi,CASH
4,5911206E,18-10-2023,12:45:58,Wates,Balibul Wates,"Teh Panas/Es, Reguler",2,0,5000,10000,Normal,0,0,0,10000,Transaksi,CASH


In [3]:
#Memisahkan kolom produk menjadi Nama Produk dan Jenis Produk
# Pisahkan kolom "Produk" menjadi "Nama Produk" dan "Jenis Produk"
df[['Nama Produk', 'Jenis Produk']] = df['Produk'].str.split(', ', expand=True, n=1)
# Hapus kolom "Produk"
df = df.drop('Produk', axis=1)
# Ubah urutan kolom sesuai dengan keinginan
df = df[['No. Struk', 'Tanggal', 'Jam', 'Nama Outlet', 'Nama Kasir', 'Nama Produk', 'Jenis Produk', 'Jumlah Produk', 'Jumlah Dibatalkan', 'Harga Per Produk', 'Subtotal', 'Tipe Harga', 'Diskon Produk', 'Diskon Transaksi', 'Pajak', 'Total', 'Status', 'Metode Pembayaran']]
# Tampilkan hasil
print(df[['Nama Produk', 'Jenis Produk']])
df.head()

                 Nama Produk Jenis Produk
0       Sate Solo 10 Campur       Reguler
1                  Tongseng       Reguler
2      Tengkleng Masak/Segar      Reguler
3                Sop Kambing      Reguler
4               Teh Panas/Es      Reguler
...                      ...          ...
25844           Teh Panas/Es      Reguler
25845     Lemon Tea Panas/Es      Reguler
25846             Nasi Putih      Reguler
25847             Pio Puding      Reguler
25848          Snack Brincil      Reguler

[25849 rows x 2 columns]


,No. Struk,Tanggal,Jam,Nama Outlet,Nama Kasir,Nama Produk,Jenis Produk,Jumlah Produk,Jumlah Dibatalkan,Harga Per Produk,Subtotal,Tipe Harga,Diskon Produk,Diskon Transaksi,Pajak,Total,Status,Metode Pembayaran
0,5911206E,18-10-2023,12:45:58,Wates,Balibul Wates,Sate Solo 10 Campur,Reguler,6,0,60000,360000,Normal,0,0,0,360000,Transaksi,CASH
1,5911206E,18-10-2023,12:45:58,Wates,Balibul Wates,Tongseng,Reguler,1,0,35000,35000,Normal,0,0,0,35000,Transaksi,CASH
2,5911206E,18-10-2023,12:45:58,Wates,Balibul Wates,Tengkleng Masak/Segar,Reguler,1,0,35000,35000,Normal,0,0,0,35000,Transaksi,CASH
3,5911206E,18-10-2023,12:45:58,Wates,Balibul Wates,Sop Kambing,Reguler,1,0,35000,35000,Normal,0,0,0,35000,Transaksi,CASH
4,5911206E,18-10-2023,12:45:58,Wates,Balibul Wates,Teh Panas/Es,Reguler,2,0,5000,10000,Normal,0,0,0,10000,Transaksi,CASH


Membuat dataframe untuk memisahkan berdasar Shift

In [4]:
# Konversi kolom Tanggal ke format DateTime
df['Tanggal'] = pd.to_datetime(df['Tanggal'])
# Konversi kolom Jam ke format DateTime (Hanya jam dan menit)
df['Jam'] = pd.to_datetime(df['Jam'], format='%H:%M:%S').dt.time
# Gabungkan kolom 'Tanggal' dan 'Jam' ke dalam satu kolom DateTime
df['Tanggal'] = pd.to_datetime(df['Tanggal'].astype(str) + ' ' + df['Jam'].astype(str))
# Definisikan fungsi untuk mendapatkan shift
def get_shift(time):
    if 8 <= time.hour < 14:
        return 1
    elif 14 <= time.hour < 22:
        return 2
    else:
        return 0  # Shift 0 untuk transaksi di luar jam kerja
# Terapkan fungsi shift ke DataFrame
df['Shift'] = df['Tanggal'].apply(get_shift)
# Buat DataFrame baru dengan data transaksi setiap shift
shift_data = df.groupby(['Tanggal', 'Shift']).agg({'No. Struk': 'nunique', 'Subtotal': 'sum'}).reset_index()
shift_data.columns = ['Tanggal', 'Shift', 'Jumlah Transaksi', 'Subtotal']
print(shift_data[shift_data['Shift'] == 1])
# Tampilkan hasil
print(shift_data)

C:\Users\kleop\AppData\Local\Temp\ipykernel_23800\1763676286.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Tanggal'] = pd.to_datetime(df['Tanggal'])


                 Tanggal  Shift  Jumlah Transaksi  Subtotal
0    2023-07-18 11:09:50      1                 1    301000
1    2023-07-18 12:24:55      1                 1    124000
2    2023-07-18 12:40:52      1                 1    352000
3    2023-07-18 13:00:23      1                 1     45000
4    2023-07-18 13:01:52      1                 1    437000
...                  ...    ...               ...       ...
4796 2023-10-18 12:28:06      1                 1    109000
4797 2023-10-18 12:28:48      1                 1     95000
4798 2023-10-18 12:41:08      1                 1     89000
4799 2023-10-18 12:44:35      1                 1    171000
4800 2023-10-18 12:45:58      1                 1    506000

[1633 rows x 4 columns]
                 Tanggal  Shift  Jumlah Transaksi  Subtotal
0    2023-07-18 11:09:50      1                 1    301000
1    2023-07-18 12:24:55      1                 1    124000
2    2023-07-18 12:40:52      1                 1    352000
3    2023-07-18

In [5]:
# Konversi kembali datetime "Tanggal" ke date
shift_data["Tanggal"] = shift_data["Tanggal"].dt.date

# Sum "Subtotal" berdasarkan "Tanggal" dan "Shift"
shift_data = shift_data.groupby(["Tanggal", "Shift"]).agg({"Subtotal": "sum"}).reset_index()
shift_data.columns = ["Tanggal", "Shift", "Subtotal"]

print(shift_data[shift_data["Shift"] == 1])
shift_data.head()

        Tanggal  Shift  Subtotal
0    2023-07-18      1   1577000
2    2023-07-19      1   3111000
4    2023-07-20      1   2716000
6    2023-07-21      1   4123750
8    2023-07-22      1   5798500
..          ...    ...       ...
177  2023-10-14      1   7921000
179  2023-10-15      1   3706000
181  2023-10-16      1   3306000
183  2023-10-17      1   2663250
185  2023-10-18      1   2401000

[93 rows x 3 columns]


,Tanggal,Shift,Subtotal
0,2023-07-18,1,1577000
1,2023-07-18,2,5343000
2,2023-07-19,1,3111000
3,2023-07-19,2,3928000
4,2023-07-20,1,2716000


In [6]:
#mengurangi subtotal
shift_data = shift_data[shift_data['Subtotal'] != 0]
shift_data.describe()
print(shift_data)

        Tanggal  Shift  Subtotal
0    2023-07-18      1   1577000
1    2023-07-18      2   5343000
2    2023-07-19      1   3111000
3    2023-07-19      2   3928000
4    2023-07-20      1   2716000
..          ...    ...       ...
181  2023-10-16      1   3306000
182  2023-10-16      2   4325000
183  2023-10-17      1   2663250
184  2023-10-17      2   6767000
185  2023-10-18      1   2401000

[186 rows x 3 columns]


In [7]:
# Mengubah kolom 'Tanggal' menjadi tipe datetime
shift_data['Tanggal'] = pd.to_datetime(shift_data['Tanggal'])

# Fungsi untuk memplot distribusi Subtotal
def plot_histogram(bin_width):
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Mengkonversi Subtotal ke jutaan
    shift_data['Subtotal (Jutaan)'] = shift_data['Subtotal'] / 1e6

    # Plot distribusi Subtotal dalam jutaan
    sns.histplot(shift_data['Subtotal (Jutaan)'], kde=True, color='skyblue', ax=ax, bins=bin_width)
    ax.set_title(f'Distribusi Subtotal dalam Jutaan (Bin Width: {bin_width})', size=12)
    ax.set_xlabel('Subtotal (Jutaan)', size=10)
    ax.set_ylabel('Frequency', size=10)

# Membuat slider untuk mengatur bin width
bin_width_slider = widgets.IntSlider(value=10, min=1, max=50, step=1, description='Bin Width')

# Menghubungkan slider dengan fungsi plot_histogram
interact(plot_histogram, bin_width=bin_width_slider)

plt.tight_layout()
plt.show()

interactive(children=(IntSlider(value=10, description='Bin Width', max=50, min=1), Output()), _dom_classes=('w…

<Figure size 640x480 with 0 Axes>

In [8]:
shift_data['Tanggal'] = pd.to_datetime(shift_data['Tanggal'])

# Fungsi untuk memplot distribusi Subtotal
def plot_scatter(date_format, shift):
    fig, ax = plt.subplots(figsize=(12, 6))  # Mengatur lebar yang lebih besar
    
    # Filter data berdasarkan shift
    filtered_data = shift_data[shift_data['Shift'] == shift]
    
    # Plot distribusi Subtotal dengan tanggal di sumbu x dan Subtotal di sumbu y
    ax.scatter(filtered_data['Tanggal'], filtered_data['Subtotal'], color='skyblue', label=f'Shift {shift}')
    ax.set_title(f'Distribusi Subtotal per Tanggal', size=12)
    ax.set_xlabel('Tanggal', size=10)
    ax.set_ylabel('Subtotal', size=10)
    
    date_format = mdates.DateFormatter(date_format)
    ax.xaxis.set_major_formatter(date_format)
    
    # Mengatur aspek sumbu x agar lebih panjang dibandingkan dengan sumbu y
    ax.set_box_aspect(1/6)  # Sesuaikan angka sesuai dengan kebutuhan
    
    # Menambahkan legenda
    ax.legend()

# Membuat pilihan untuk format tanggal
# Membuat pilihan untuk format tanggal
date_format_dropdown = widgets.Dropdown(
    options=['%Y-%m-%d', '%d-%m-%Y'],
    value='%Y-%m-%d',
    description='Date Format',
)


# Membuat pilihan untuk shift
shift_dropdown = widgets.Dropdown(
    options=[1, 2],
    value=1,
    description='Shift',
)

# Menghubungkan dropdown dengan fungsi plot_scatter
interact(plot_scatter, date_format=date_format_dropdown, shift=shift_dropdown)

plt.tight_layout()
plt.show()

interactive(children=(Dropdown(description='Date Format', options=('%Y-%m-%d', '%d-%m-%Y'), value='%Y-%m-%d'),…

<Figure size 640x480 with 0 Axes>

In [9]:
# Mengubah kolom 'Tanggal' menjadi tipe datetime
shift_data['Tanggal'] = pd.to_datetime(shift_data['Tanggal'])

# Fungsi untuk memplot distribusi Subtotal dalam jutaan berdasarkan Shift
def plot_histogram(bin_width, shift):
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Filter data berdasarkan Shift
    filtered_data = shift_data[shift_data['Shift'] == shift].copy()  # Membuat salinan data

    # Mengkonversi Subtotal ke jutaan
    filtered_data['Subtotal (Jutaan)'] = filtered_data['Subtotal'] / 1e6


    # Plot distribusi Subtotal dalam jutaan
    sns.histplot(filtered_data['Subtotal (Jutaan)'], kde=True, color='skyblue', ax=ax, bins=bin_width)
    ax.set_title(f'Distribusi Subtotal dalam Jutaan (Shift {shift}, Bin Width: {bin_width})', size=12)
    ax.set_xlabel('Subtotal (Jutaan)', size=10)
    ax.set_ylabel('Frequency', size=10)

# Membuat slider untuk mengatur bin width
bin_width_slider = widgets.IntSlider(value=10, min=1, max=50, step=1, description='Bin Width')

# Membuat dropdown untuk memilih Shift
shift_dropdown = widgets.Dropdown(
    options=[1, 2],
    value=1,
    description='Shift',
)

# Menghubungkan slider dan dropdown dengan fungsi plot_histogram
interact(plot_histogram, bin_width=bin_width_slider, shift=shift_dropdown)

plt.tight_layout()
plt.show()

interactive(children=(IntSlider(value=10, description='Bin Width', max=50, min=1), Dropdown(description='Shift…

<Figure size 640x480 with 0 Axes>

In [10]:
data_perhari=shift_data
data_perhari['Hari'] = shift_data['Tanggal'].dt.strftime('%A')
data_perhari.head()

,Tanggal,Shift,Subtotal,Subtotal (Jutaan),Hari
0,2023-07-18,1,1577000,1.577,Tuesday
1,2023-07-18,2,5343000,5.343,Tuesday
2,2023-07-19,1,3111000,3.111,Wednesday
3,2023-07-19,2,3928000,3.928,Wednesday
4,2023-07-20,1,2716000,2.716,Thursday


In [11]:
# Mengubah kolom 'Tanggal' menjadi tipe datetime
data_perhari['Tanggal'] = pd.to_datetime(data_perhari['Tanggal'])

# Menambahkan kolom 'Hari' yang berisi nama hari
data_perhari['Hari'] = data_perhari['Tanggal'].dt.strftime('%A')

# Fungsi untuk memplot distribusi Subtotal berdasarkan hari
def plot_histogram(day, shift):
    filtered_data = data_perhari[(data_perhari['Hari'] == day) & (data_perhari['Shift'] == shift)]
    
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.histplot(filtered_data['Subtotal'], kde=True, color='skyblue', ax=ax, bins=10)
    ax.set_title(f'Distribusi Subtotal ({day}, Shift {shift})', size=12)
    ax.set_xlabel('Subtotal', size=10)
    ax.set_ylabel('Frequency', size=10)

# Membuat dropdown untuk memilih hari
day_dropdown = widgets.Dropdown(
    options=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    value='Monday',
    description='Hari',
)

# Membuat dropdown untuk memilih shift
shift_dropdown = widgets.Dropdown(
    options=[1, 2],
    value=1,
    description='Shift',
)

# Menghubungkan dropdown dengan fungsi plot_histogram
interact(plot_histogram, day=day_dropdown, shift=shift_dropdown)

plt.tight_layout()
plt.show()

interactive(children=(Dropdown(description='Hari', options=('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Fri…

<Figure size 640x480 with 0 Axes>

In [12]:
# Mengubah kolom 'Tanggal' menjadi tipe datetime
shift_data['Tanggal'] = pd.to_datetime(shift_data['Tanggal'])

# Fungsi untuk memplot scatter plot
def plot_scatter(day, shift):
    # Filter data berdasarkan hari dan shift
    filtered_data = shift_data[(shift_data['Tanggal'].dt.strftime('%A') == day) & (shift_data['Shift'] == shift)]

    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot scatter plot dengan tanggal di sumbu x dan Subtotal di sumbu y
    ax.scatter(filtered_data['Tanggal'], filtered_data['Subtotal'], color='skyblue', label=f'{day}, Shift {shift}')
    ax.set_title(f'Scatter Plot Subtotal ({day}, Shift {shift})', size=12)
    ax.set_xlabel('Tanggal', size=10)
    ax.set_ylabel('Subtotal', size=10)
    
    date_format = mdates.DateFormatter('%Y-%m-%d')
    ax.xaxis.set_major_formatter(date_format)
    
    # Mengatur aspek sumbu x agar lebih memanjang dibandingkan dengan sumbu y
    ax.set_box_aspect(1/6)
    
    # Menambahkan legenda
    ax.legend()

# Membuat dropdown untuk memilih hari
day_dropdown = widgets.Dropdown(
    options=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    value='Monday',
    description='Hari',
)

# Membuat dropdown untuk memilih shift
shift_dropdown = widgets.Dropdown(
    options=[1, 2],
    value=1,
    description='Shift',
)

# Menghubungkan dropdown dengan fungsi plot_scatter
interact(plot_scatter, day=day_dropdown, shift=shift_dropdown)

plt.tight_layout()
plt.show()

interactive(children=(Dropdown(description='Hari', options=('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Fri…

<Figure size 640x480 with 0 Axes>